In [1]:
# Setup the Jupyter version of Dash 
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "AACuser"
password = "Juwon"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# Drop the MongoDB '_id' column to avoid issues
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
logo_img = html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '300px', 'height': 'auto'})

app = JupyterDash(__name__)  # Initialize the app

app.layout = html.Div([
    html.Center(html.B(html.H1("Juwon's Client-Server Authentication"))),
    html.Hr(),
    
    # Place the logo in the layout
    logo_img,
    
    # Interactive filtering options
    html.Div([
        dcc.Dropdown(
            id='filter-breed',
            options=[{'label': breed, 'value': breed} for breed in df['breed'].unique()],
            placeholder="Select a breed",
            multi=True
        ),
        dcc.Checklist(
            id='filter-adopted',
            options=[
                {'label': 'Adopted', 'value': 'Yes'},
                {'label': 'Not Adopted', 'value': 'No'}
            ],
            value=[],
            inline=True
        )
    ], style={'width': '50%', 'padding': '20px'}),
    
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
    ),
    
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id', 'data'),
              [Input('filter-breed', 'value'),
               Input('filter-adopted', 'value')])
def update_dashboard(selected_breeds, selected_adopted):
    filtered_df = df
    
    # Filter by breed
    if selected_breeds:
        filtered_df = filtered_df[filtered_df['breed'].isin(selected_breeds)]
    
    # Filter by adopted status
    if selected_adopted:
        filtered_df = filtered_df[filtered_df['adopted'].isin(selected_adopted)]
    
    return filtered_df.to_dict('records')

# Display the breeds of animal based on quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    #FIX ME #### add code for chart of your choice (e.g. pie chart) #
    return [
        dcc.Graph(            
            figure=px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None or index is None:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

app.run_server(debug=True)


connection Successful
Dash app running on http://127.0.0.1:30879/


In [ ]:
# Setup the Jupyter version of Dash  
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os
import argparse
from tabulate import tabulate

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "AACuser"
password = "Juwon"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# Drop the MongoDB '_id' column to avoid issues
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Place the HTML image tag in the line below into the app.layout code according to your design
logo_img = html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '300px', 'height': 'auto'})

# Dash app setup
app = JupyterDash(__name__)  # Initialize the app

app.layout = html.Div([
    html.Center(html.B(html.H1("Juwon's Client-Server Authentication"))),
    html.Hr(),
    
    # Place the logo in the layout
    logo_img,
    
    # Interactive filtering options
    html.Div([
        dcc.Dropdown(
            id='filter-breed',
            options=[{'label': breed, 'value': breed} for breed in df['breed'].unique()],
            placeholder="Select a breed",
            multi=True
        ),
        dcc.Checklist(
            id='filter-adopted',
            options=[
                {'label': 'Adopted', 'value': 'Yes'},
                {'label': 'Not Adopted', 'value': 'No'}
            ],
            value=[],
            inline=True
        )
    ], style={'width': '50%', 'padding': '20px'}),
    
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
    ),
    
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
#
